In [2]:
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import nn
from torch.cuda import is_available
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchinfo import summary
import modules.data_setup as data_setup
import modules.engine as engine
import os


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

(train_dir: str, test_dir: str, train_transform: torchvision.transforms.transforms.Compose = None, test_transform: torchvision.transforms.transforms.Compose = None, batch_size: int = 32, num_workers: int = 10)
Device: cpu


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
version = torch.__version__
print(f"PyTorch Version: {version}")

PyTorch Version: 2.8.0


In [4]:
import requests
from pathlib import Path
import zipfile

data_path = Path('data')
image_path = data_path / 'pizza_steak_sushi_20_percent'

if image_path.is_dir():
    print(f'{image_path} exists')
else:
    print(f'{image_path} does not exist, creating...')
    image_path.mkdir(parents=True, exist_ok=True)

    with open(data_path / 'pizza_steak_sushi_20_percent.zip','wb') as f:
         request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip")
         print('Downloading pizza_steak_sushi_20_percent data')
         f.write(request.content)
    with zipfile.ZipFile(data_path / 'pizza_steak_sushi_20_percent.zip','r') as zip_ref:
        print("Extracting pizza_steak_sushi_20_percent.zip")
        zip_ref.extractall(image_path)

    os.remove(data_path / 'pizza_steak_sushi_20_percent.zip')

data/pizza_steak_sushi_20_percent exists


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #device agnostic setting

In [6]:
def set_seed(seed:int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [7]:
train_dir = image_path/'train'
test_dir = image_path/'test'
normalize = transforms.Normalize(
    mean=[0.485,0.456,0.406],
    std=[0.229,0.224,0.225]
)
simple_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])
print(simple_transform)


Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [8]:
data_augmentation_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    normalize
])
print(data_augmentation_transform)

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    TrivialAugmentWide(num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [10]:
train_dataloader,test_dataloader,class_names = data_setup.create_dataloaders(
                                                train_dir=train_dir,
                                                test_dir=test_dir,
                                                train_transform=simple_transform,
                                                test_transform=simple_transform,
                                                batch_size=32)
train_dataloader,test_dataloader,class_names

(<torch.utils.data.dataloader.DataLoader at 0x12794ef60>,
 ['pizza', 'steak', 'sushi'])

In [11]:
train_dataloader_with_aug,test_dataloader,class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    train_transform=data_augmentation_transform,
    test_transform=simple_transform,
    batch_size=32
)
train_dataloader_with_aug,test_dataloader,class_names

(<torch.utils.data.dataloader.DataLoader at 0x127c1d580>,
 ['pizza', 'steak', 'sushi'])